# Cell Counting
by David Piena, Waleed, and Jorge Fuentes

## Table of contents
1. [File Helpers](#file)
2. [Cell Counting](#cell)
3. [Main Code](#main)
4. [Debugging](#debug)
5. [Testing](#test)

In [2]:
import os, matplotlib, sys
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import xlwt, mpld3, xlrd #pandas stuff 
from skimage import measure
from scipy import ndimage, misc, stats 
from tqdm import tqdm
matplotlib.style.use('ggplot')

## File Helpers <a name="file"></a>
These functions build off the os library to navigate the file structure

In [3]:
def filterdirectory(path,extension):
    """return list of files under the directory given by the path that ends with the extension"""
    files = [file for file in os.listdir(path) if file.lower().endswith(extension) and file[0] !='.']
    return files

In [4]:
def mkdirsafe (newpath):
    """make directory if it doesn't already exist"""
    if not os.path.exists(newpath): os.makedirs(newpath)

In [5]:
def useImgDir(path):
    """switch to img path and create all the needed folders"""
    os.chdir(path)
    mkdirsafe('panda')
    mkdirsafe('3d')
    mkdirsafe('arrays')
    mkdirsafe('ExcelSheets')
    mkdirsafe('DebugImages')

## Cell Counting <a name="cell"></a>
These functions do all the image analysis including object detection and region counting using the Allen Brain Atlas

In [6]:
class detectionobject:
    def detectlabels (self,array):
        """returns array where identical pixels are given same label"""
        labeled, num = measure.label(array, return_num=True)
#         print("NUM", num)
        return labeled
    
    def detectobjects (self,labels):
        """returns the minimal parallelepiped that contains all of one label in slices"""
        objects = ndimage.measurements.find_objects(labels)
        return objects

    def arealist (self,array,objects):
        """returns list of the sizes of objects in same order"""
        areas = []
        [areas.append(array[obj].size) for obj in objects]
        return areas
    
    def __init__(self, array):
        """creates labels, objects, list of areas, largest area slice, and array largest area"""
        self.labeled = self.detectlabels (array)
        self.objects = self.detectobjects(self.labeled)
        self.objectareas = self.arealist(array, self.objects)
        #== statement returns an boolean array where the only spot that is true is the max area
        #[0][0] gets location of max(since its the first/only nonzero) so you get index of object of largest area
        self.largestobjectslice = self.objects[np.nonzero(np.array(self.objectareas)== np.array(self.objectareas).max())[0][0]]
        #gets actual array values of object slice
        self.largestobjectarray = array[self.largestobjectslice]

In [7]:
 def brightValues(imagearray, sd_from_mean=4.5):
    #thresh is a boolean array mirroring the image true if the pixel value is 4.5 std's bigger than the mean
    thresh = imagearray>(imagearray.mean() + sd_from_mean*imagearray.std())
    if thresh.sum()<30: #if there were only a few numbers over thresh then lower standards
        thresh = imagearray>(imagearray.mean() + sd_from_mean-1*imagearray.std())
    return thresh

In [8]:
def filterObjects(labels, min_size_percent, max_size_percent):
    """returns the labels and the objects within the threshold of size(judged by number of elements)"""
    
    #find slices for each group of pixels
    objects = ndimage.measurements.find_objects(labels)
    
    #only get objects sized within bounds that exist(returns none when numbered labels missing) 
    filtered_objects = []
    for obj in objects:
        if obj:
            if labels[obj].size > min_size_percent*labels.size and labels[obj].size < max_size_percent*labels.size:
                filtered_objects.append(obj)
    
    #np.set_printoptions(threshold='nan')#this line messes up printing should be threshold = nan apparently
    #FIND THE SIGNIFICANCE OF MAKING ALL THE NON OBJECTS NOW 0 
    return labels, filtered_objects 

In [9]:
def _zoom2Large (largearray,smallarray):
    """Zoom the small array to the size of the large array"""
    largearrayshape = np.float64(largearray.shape)
    smallarrayshape = np.float64 (smallarray.shape)
    zoomfactor = (largearrayshape [0]/smallarrayshape[0],largearrayshape[1]/smallarrayshape[1])
    zoomfactor = zoomfactor #delete this line
    zoomedsmall = ndimage.zoom(smallarray, order=0, zoom = zoomfactor)
    return zoomedsmall

In [10]:
def allencomparisonarray (sectionnumber, imagearray, Allen_detect_annotation_path='/home/dfpena/Documents/P56_Mouse_annotation/' ):
    """Read the sectionnumber from the Allen and zoom it to the size of the imagearray"""
    #read the int32 array from the panda
    Allendetected = pd.read_pickle(Allen_detect_annotation_path +'Allen_detected_annotation.panda').values[sectionnumber][0]
    largeallen = _zoom2Large(imagearray,Allendetected)
    return largeallen

In [11]:
def filterAllenMapby(allenmap,conversionkeyword,startkeyword,identifierIter):
    """Get the conversionKeyword for the allenmap panda rows where the startkeyword of that row = the identifiers"""
    #allenmap[[s..]].v..==i.. gives a boolean array identifying the allemap rows where the startkeyword = the identifies
    #[[s..]] double brackets probably unneccessary 
    #np.nonzero(...)[0][0] gives the index of the row of the ...
    #allenmap[conv...].v...[^] gives the allenmap's conversionkeyword value of ^ index
    filtered = [allenmap[conversionkeyword].values[np.nonzero(allenmap[[startkeyword]].values==identifier)[0][0]] for identifier in identifierIter]
    return filtered

In [12]:
def generatecelllocationarray(structureslist,sectionnumber,Regionnames,parentid,parentname):
    """Create a pandas dataframe with a row of the section number, structurelist[:,1&2], Regionnames, & parentid"""
    #parentname seems unused
    #Side is 0 if left, 1 if right side
    cellmap = pd.DataFrame({'SectionNumber': np.array([sectionnumber] * len(structureslist)),
                            'StructureID': structureslist[:,1],
                            'Side': structureslist[:,2],
                            'StructureName': Regionnames,
                            'parent_structure_id': parentid,
                            
                            
                           })
    return cellmap

In [13]:
def enumerateRegions(objects,ResizedAllen,Sectionnumber,allenpandamappath ='/home/dfpena/Documents/P56_Mouse_annotation/P56_Mouse_annotation/'):
    """generate a pandas DataFrame recording the info about the objects from the image overlaid on the resized allen"""
    allenmap = pd.read_pickle(allenpandamappath + 'Allen_Lookup.panda')
    #lookup is a panda dataframe with atlas_id,id,name,ontology_id and parent structure 
    #recall objects are slices
    AllenRegion=[]  
    #obj[1].start gets the lowest index of the bounding box for the x/width axis 
    #stats... takes the mode(probably id naming region) of the object slices from the processing image overlayed on allen([0][0] gives single top mode)
    #AllenRegion is composed of a 2D list which each sublist is 3 elements corresponding to the start, mode, and -1 
    [AllenRegion.append([obj[1].start,stats.mode(ResizedAllen[obj].flatten())[0][0].astype('int'),-1]) for obj in objects];
    AllenRegion= np.array(AllenRegion)
    #Allen Region = all 3 elements of the list where the mode is greater than 0 and not equal to 8
    AllenRegion= AllenRegion[AllenRegion[:,1] > 0] #TODO: THIS SOMETIMES gets, IndexError: too many indices for array
    AllenRegion= AllenRegion[AllenRegion[:,1] != 8]
    #range number of rows left
    selector_range = np.arange(AllenRegion.shape[0])
    #bool of rows where the object starts in the image below the width/2 
    bool_selector =  (AllenRegion[:,0] < ResizedAllen.shape[1]/2)
    #set third element(currently -1) of AllenRegion to 0 if object starts in image below the width/2, and 1 otherwise
    AllenRegion[selector_range[bool_selector ],2]= 0
    AllenRegion[selector_range[np.invert(bool_selector) ],2] = 1
    #using the AllenRegions, the variable names are well named
    Regionnames = filterAllenMapby(allenmap,'name','id',AllenRegion[:,1])
    parentid = filterAllenMapby(allenmap,'parent_structure_id','id',AllenRegion[:,1])
    parentname = filterAllenMapby(allenmap,'name','id',parentid)
    grandparentid = filterAllenMapby(allenmap,'parent_structure_id','id',parentid)
    grandparentname = filterAllenMapby(allenmap,'name','id',grandparentid)
    cellmap = generatecelllocationarray(AllenRegion,Sectionnumber,Regionnames,parentid,parentname)
    return cellmap

In [14]:
def process_images(ipath, iterator, allenlibpath, sd_from_mean=4.5, min_size_percent=.00000003, max_size_percent=.0000009):
    """
    Find objects in the image and overlays the corresponding allen image. Then, create and save panda dataframe.
    
    :param string ipath: path to image
    :param int iterator: unique value for this batch process(start Debug_Images name)
    :param string allenlibpath: path to allen library with brains
    :param int sd_from_mean: standard deviations above mean that counts as bright
    """
    print("Processing ", iterator, ipath)
    try:
        #get sectionnumber from number before first _ in name
        Sectionnumber = int(ipath.split('_')[0])
        image = imageio.imread(ipath)

        #get and save brightValues
        brightMask = brightValues(image, sd_from_mean)
        image[~brightMask] = 0
        image[brightMask] = 255
        imageio.imwrite('DebugImages/'+str(iterator)+'_'+str(Sectionnumber)+'_'+'brightValues.jpg', image)

        #turn boolean mask into "labeled" array with unique numbers noting each group/adajecent values
        labeled = detectionobject(brightMask).labeled 

        #TODO: Actually use detection object for this intermidate steps, add lower and upper dependent on percent of size
        #get objects(tuples of slices indicating rows and columns (start, exclusive end) for all objects)
        labeled, objects = filterObjects(labeled, min_size_percent, max_size_percent)

        #TODO: Save debug image of labeled, maybe by randomizing the colors

        #name and save allen image corresponding to the sectionnumber zoomed to the size of the image
        Allen = allencomparisonarray(Sectionnumber,image,allenlibpath) #allen seems to be grayscale 
        #TODO: Save debug image

        #generatecelllocationarray cellmap
        cellmap =enumerateRegions(objects,Allen,Sectionnumber,allenlibpath)
        #save the objects and cellmap, print and return the iterator and the path to the image
        cellmap.to_pickle('panda/'+ str(iterator) + '_' +str(Sectionnumber)+'.panda')
        np.savez('3d/'+str(iterator)+'_'+str(Sectionnumber),objects )
        return (iterator, ipath)
    except ValueError:
        print('No cells could be detected')
        print(iterator, ipath)
        return (iterator, ipath)

In [15]:
def Collect_Pandas(directorylist):
    """Create single container of all the pickled pandas in the directory"""
    container = pd.DataFrame()
    container = container.append([pd.read_pickle(file) for file in directorylist])
    return container

In [16]:
def mkexcel( path_to_pandas, animalname): #need xls lib
    """gather all the panda dataframes and create excel sheets and pickles total and for each side"""
    os.chdir(path_to_pandas)
    directory = filterdirectory(os.curdir,".panda")

    #get all the pandas and divide them in the left and right side
    Brain = Collect_Pandas(directory)
    LateralizedDF = Brain.groupby(['Side'])
    LeftDF = LateralizedDF.get_group(0)
    RightDF = LateralizedDF.get_group(1)
    #count the unique structure names on each side and total 
    TotalCountsDF = pd.DataFrame({'Cells':Brain.StructureName.value_counts()})
    LeftCountsDF = pd.DataFrame({'Cells':LeftDF.StructureName.value_counts()})
    RightCountsDF = pd.DataFrame({'Cells':RightDF.StructureName.value_counts()})
    
    os.chdir('..')
    mkdirsafe('ExcelSheets')
    #Raw
    Brain.to_pickle('ExcelSheets/'+animalname +'_total.compile')
    Brain.to_excel('ExcelSheets/'+animalname +'_RawTotal.xls')
    LeftDF.to_excel ('ExcelSheets/'+animalname +'_RawLeft.xls')
    RightDF.to_excel ('ExcelSheets/'+animalname +'_RawRight.xls')

    #Compiled
    TotalCountsDF.to_excel ('ExcelSheets/'+animalname +'_TotalFreq.xls')
    LeftCountsDF.to_excel ('ExcelSheets/'+animalname +'_LeftFreq.xls')
    RightCountsDF.to_excel ('ExcelSheets/'+animalname +'_RightFreq.xls')

## Main Code<a name="main"></a>
Remember your images should have a sectionnumber before '_' to indicate they are part of the same specimen. And the images should already be only green(greyscale green), cropped, and straigtened, see preprocessing to do this automatically.

Change the arguments below then run the next few blocks. 

In [21]:
#change these arguments 
animalname = '18'
path_to_images = "/Users/jfuentes/Projects/Allen-Brain-Analysis/idealImages/2495/preprocessed/" #use the preprocessed folder if you used that code
path_to_allenlib = "/Users/jfuentes/Projects/Allen-Brain-Analysis/Allen/"
sd_from_mean = 4.5 #the number of standard deviations from the mean that cells to be noted are
min_size_percent = .00000003 #min size of cells in relation to image size
max_size_percent = .0000009 #max size of cells in relation to image size

#for command line running in script version
if len(sys.argv) == 4:
    print("Using command line arguments")
    animalname = sys.argv[1]
    path_to_images = sys.argv[2]
    path_to_allenlib = sys.argv[3]

In [22]:
path_to_figurearray= path_to_images + 'arrays/'
path_to_panda = path_to_images +'panda/'

useImgDir(path_to_images)
directory = filterdirectory(os.curdir,".jpg")
print(directory)

i=0
for img in tqdm(directory):
    process_images(img, i, path_to_allenlib, sd_from_mean, min_size_percent, max_size_percent)
    i+=1
    
mkexcel(path_to_panda, animalname)

  0%|          | 0/61 [00:00<?, ?it/s]

['165_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{4}.jpg', '185_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{6}.jpg', '250_out_{Wendy}{012115}{2495-3}{Jan 21 15, 165035}{ImmunoFluorescentHighRes}{4}.jpg', '405_out_{Wendy}{012115}{2495-5}{Jan 21 15, 165333}{ImmunoFluorescentHighRes}{6}.jpg', '374_out_{Wendy}{012115}{2495-5}{Jan 21 15, 165333}{ImmunoFluorescentHighRes}{3}.jpg', '305_out_{Wendy}{012115}{2495-4}{Jan 21 15, 165204}{ImmunoFluorescentHighRes}{11}.jpg', '125_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{1}.jpg', '105_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{3}.jpg', '153_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{14}.jpg', '230_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{11}.jpg', '285_out_{Wendy}{012115}{2495-3}{Jan 21 15, 165035}{ImmunoFluorescentHighRes}{10}.jpg', '226_out_{Wendy}{012115}{2495-2}{Jan 2

  2%|▏         | 1/61 [00:07<07:52,  7.88s/it]

Processing  1 185_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{6}.jpg


  3%|▎         | 2/61 [00:15<07:26,  7.57s/it]

...

 66%|██████▌   | 40/61 [04:48<02:31,  7.21s/it]/anaconda2/envs/brainLab/lib/python3.6/site-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (108732680 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Processing  40 261_out_{Wendy}{012115}{2495-3}{Jan 21 15, 165035}{ImmunoFluorescentHighRes}{1}.jpg


 67%|██████▋   | 41/61 [04:56<02:24,  7.24s/it]

Processing  41 137_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{7}.jpg


IndexError: too many indices for array

## Debugging<a name="debug"></a>
These functions use the premade pandas to make pretty infographics

In [ ]:
def mkmpld3figs (path_to_ExcelSheetsfolder,animalname): #need mpld3 and xlrd lib
    """make a cool html bar graph of excel sheet"""
    import mpld3
    from mpld3 import plugins
    os.chdir(path_to_ExcelSheetsfolder)
    TotalCountsDF = pd.read_excel(animalname+'_TotalFreq.xls')
    fig = plt.figure(figsize=(10,5))
    bars = plt.bar(np.arange(TotalCountsDF.size), TotalCountsDF.values)
    for i, bar in enumerate(bars.get_children()):
        tooltip = mpld3.plugins.LineLabelTooltip(bar, label=str(TotalCountsDF.index[i]))
        mpld3.plugins.connect(plt.gcf(), tooltip)
    plt.xlim(0,50)
    plt.title('Cell Detection for : Subject {0}'.format(animalname))
    plt.ylabel('Cell Count')
    plt.xlabel('Region number (Scroll over Bar for Region Name)')

    a = mpld3.fig_to_html(fig)


    Html_file= open("{0}.html".format(animalname),"w")
    Html_file.write(a)
    Html_file.close()
    plt.savefig('{0}.svg'.format(animalname))

In [ ]:
def mkdebug_fig(folder,name,fig, alphav=1):
    """save fig to folder/name"""
    mkdirsafe(folder)
    plt.imshow(fig,alpha=alphav)
    plt.savefig(folder+'/'+name)

In [ ]:
def figure_from_pickle(picklepath):
    """Make a image from all the saved arrays"""
    import gc
    #npz's load a dict 
    figure = np.load(picklepath)['arr_0']
    name = picklepath.split('.')
    mkdebug_fig('DebugImages',(name[0]),figure)
    plt.close()
    #cool garbage collection
    gc.collect()

In [ ]:
def easyRun(animalname, path_to_images, path_to_allenlib):
    path_to_figurearray= path_to_images + 'arrays/'

    os.chdir(path_to_images) 
    directory = filterdirectory(os.curdir,".jpg")
    print(directory)

    i=0
    for img in directory:
        process_images(img, i, path_to_allenlib)
        i+=1

    #make an excel sheet and html figures
    mkexcel(path_to_images +'panda', animalname)
    
    #beyond here is just figures and debugging stuff
    mkmpld3figs(path_to_images +'ExcelSheets',animalname)

    os.chdir(path_to_figurearray)
    pickledirectory = filterdirectory(os.curdir,".npz")
    #to debug look at these images
    for file in pickledirectory:
        figure_from_pickle(file)

In [ ]:
mkmpld3figs(path_to_images +'ExcelSheets',animalname)

os.chdir(path_to_figurearray)
pickledirectory = filterdirectory(os.curdir,".npz")
#to debug look at these images
for file in pickledirectory:
    figure_from_pickle(file)

## TESTING<a name="test"></a>

In [ ]:
test = np.array([[True, False, True],[True, True, False], [False, False, False], [True, True, True]])
detectionobject(test).labeled 